In [65]:
import pandas as pd

In [66]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['sample']
col = db['expression_data']

In [67]:
gene_info = pd.read_csv('Gene_Information.csv')
gene_info.head()

,Gene ID,Gene Name,Chromosome,Start Position,End Position
0,MEPQFQ9Z,Gene_IWVWK,chr1,761616,1578683
1,A0EACQ85,Gene_5PEJS,chr17,339598,1081528
2,HZA08PZ7,Gene_VK419,chr6,60362,1214780
3,3IBMS02U,Gene_7ZR22,chr9,276649,1577759
4,WGTF1420,Gene_N7T1C,chr17,45172,1739962


In [68]:
gene_info_dict = gene_info.to_dict(orient='records')

In [69]:
col.insert_many(gene_info_dict)

InsertManyResult([ObjectId('66706ac7d9c119c8de26046c'), ObjectId('66706ac7d9c119c8de26046d'), ObjectId('66706ac7d9c119c8de26046e'), ObjectId('66706ac7d9c119c8de26046f'), ObjectId('66706ac7d9c119c8de260470'), ObjectId('66706ac7d9c119c8de260471'), ObjectId('66706ac7d9c119c8de260472'), ObjectId('66706ac7d9c119c8de260473'), ObjectId('66706ac7d9c119c8de260474'), ObjectId('66706ac7d9c119c8de260475'), ObjectId('66706ac7d9c119c8de260476'), ObjectId('66706ac7d9c119c8de260477'), ObjectId('66706ac7d9c119c8de260478'), ObjectId('66706ac7d9c119c8de260479'), ObjectId('66706ac7d9c119c8de26047a'), ObjectId('66706ac7d9c119c8de26047b'), ObjectId('66706ac7d9c119c8de26047c'), ObjectId('66706ac7d9c119c8de26047d'), ObjectId('66706ac7d9c119c8de26047e'), ObjectId('66706ac7d9c119c8de26047f'), ObjectId('66706ac7d9c119c8de260480'), ObjectId('66706ac7d9c119c8de260481'), ObjectId('66706ac7d9c119c8de260482'), ObjectId('66706ac7d9c119c8de260483'), ObjectId('66706ac7d9c119c8de260484'), ObjectId('66706ac7d9c119c8de2604

In [70]:
expression_data = pd.read_csv('Expression_Data.csv')
expression_data.head()

,Sample ID,Gene ID,Expression Level,Condition
0,BHN1HRQWBL,MEPQFQ9Z,91.83,Cancer_Type_A
1,RO907Y9BQY,MEPQFQ9Z,85.36,Cancer_Type_A
2,TYTH27KO5M,MEPQFQ9Z,15.98,Cancer_Type_B
3,0LIP0B6TUM,MEPQFQ9Z,79.16,Cancer_Type_B
4,1RE2OFGVP7,MEPQFQ9Z,87.53,Cancer_Type_B


In [76]:
expression_data_dict = expression_data.to_dict(orient='records')
expression_data_dict[0]

{'Sample ID': 'BHN1HRQWBL',
 'Gene ID': 'MEPQFQ9Z',
 'Expression Level': 91.83,
 'Condition': 'Cancer_Type_A'}

In [72]:
for record in expression_data_dict:
    geneID = record['Gene ID']
    del record['Gene ID']
    
    col.update_one(
        {"Gene ID" : geneID},
        {"$push":{"expression":record}}
    )

In [73]:
sample_info = pd.read_csv('Sample_Information.csv')
sample_info.head()

,Sample ID,Patient ID,Collection Date,Condition,Tissue Type
0,BHN1HRQWBL,9UFVLTBI,2021-04-05,Cancer_Type_A,Blood
1,RO907Y9BQY,C9Z9VD5G,2021-07-16,Cancer_Type_A,Tissue
2,TYTH27KO5M,RZLYC6NZ,2022-06-29,Cancer_Type_B,Tissue
3,0LIP0B6TUM,29ULN150,2022-04-10,Cancer_Type_B,Blood
4,1RE2OFGVP7,Y1WFWH29,2021-09-29,Cancer_Type_B,Tissue


In [74]:
sample_info_dict = sample_info.to_dict(orient='records')

In [77]:
for record in expression_data_dict:
    geneID = record['Gene ID']
    del record['Gene ID']
    
    sampleID = record['Sample ID']
    res = sample_info[sample_info['Sample ID'] == sampleID]
    sample = res.to_dict(orient='records')[0]
    
    col.update_one(
        {'Gene ID': geneID},
        {'$push': {"sample":sample}}
    )